### Imports

In [1]:
# For Saving/Loading Files
from utils.storage import load_data,load_tokenizers

# -------------- Modelling Packages --------------
# For modeling
from keras.models import Model, Sequential
from keras.layers import Embedding, GRU, Bidirectional
from keras.layers import Input, Reshape, SpatialDropout1D, Dense, Flatten
from keras.layers import Concatenate
from keras import optimizers

# Callback Functions
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

# For Timestamping Models
import time

# For balancing class weights
from sklearn.utils import class_weight

# -------------- General Packages --------------
# Data Manipulation
import numpy as np

# Saving/Loading
import os

Using TensorFlow backend.


### Directories

In [2]:
MODEL_DIR = './models/'
SPLIT_DATA_DIR = './split_data/'
LOG_DIR = 'logs'
TOKENIZER_DIR = './tokenizers/'

### Loading the Dataset + Tokenizers

In [3]:
EN_X_train,EN_X_test,ZH_X_train,ZH_X_test,y_train,y_test = load_data(SPLIT_DATA_DIR)
t_EN,t_ZH = load_tokenizers(TOKENIZER_DIR)

### Creating the Models

In [4]:
os.makedirs(os.path.dirname(MODEL_DIR), exist_ok=True)

# -------------- Tokenizer Values --------------
EN_SENTENCE_SIZE = int(EN_X_train.shape[1]/2)
ZH_SENTENCE_SIZE = int(ZH_X_train.shape[1]/2)
en_vocab_size = len(t_EN.word_index) + 1
zh_vocab_size = len(t_ZH.word_index) + 1
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)

# -------------- TUNABLE HYPERPARAMETERS --------------
EMBED_SIZES = [ 50 ]
GRU_SIZES = [ 100 ]
GRU_LAYER_SIZES = [ 1 ]

# DROPOUT_SIZES = [(0.5,0)]
# DROPOUT_SIZES = [(0,0),(0,0.5),(0.5,0),(0.5,0.5),
#                  (0.25,0.25),(0.25,0.5),(0.5,0.25),(0.25,0),
#                  (0,0.25),(0.75,0.75),(0.75,0.5),(0.5,0.75),
#                  (0.75,0),(0,0.75),(0.75,0.25),(0.25,0.75)]

DROPOUT_SIZES = [(0,0.75),(0.75,0.25),(0.25,0.75)]

loss = 'sparse_categorical_crossentropy'
optimizer = optimizers.adam()
#optimizer = optimizers.sgd(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
metrics = ['accuracy']

epochs = 200
batch_size = 1024
PATIENCE = 3

In [ ]:
for EMBED_SIZE in EMBED_SIZES:
    for DROPOUT_SIZE in DROPOUT_SIZES:
        for GRU_SIZE in GRU_SIZES:
            for GRU_LAYERS in GRU_LAYER_SIZES:
                # -------------- MODEL NAMING --------------
                NAME = 'BiGRU-{}E-{}x{}G-{}Dropout-{}.hdf5'.format(EMBED_SIZE,
                                                                         GRU_LAYERS,GRU_SIZE,
                                                                         DROPOUT_SIZE,
                                                                         time.time())
                print('Creating {}'.format(NAME))
                MODEL_LOG_DIR = os.path.join(LOG_DIR,NAME)

                # -------------- Callbacks --------------
                # access tensorboard from the command line: tensorboard --logdir=logs/
                tensorboard = TensorBoard(log_dir=MODEL_LOG_DIR) 
                checkpointer = ModelCheckpoint(MODEL_DIR+NAME, 
                                               monitor='val_accuracy', 
                                               verbose=1, 
                                               save_best_only=True, 
                                               mode='auto')
                earlystop = EarlyStopping(monitor='val_loss', patience=PATIENCE)

                callbacks=[tensorboard,checkpointer,earlystop]

                # -------------- EN MODEL CREATION --------------
                EN_INPUT = Input(shape=(EN_SENTENCE_SIZE*2,))
                EN_MODEL = Reshape((-1,2,EN_SENTENCE_SIZE))(EN_INPUT)
                EN_MODEL = Embedding(en_vocab_size,
                                EMBED_SIZE,
                                input_shape=(2,EN_SENTENCE_SIZE),
                                trainable=True)(EN_MODEL)
                EN_MODEL = Reshape((2,EN_SENTENCE_SIZE*EMBED_SIZE,))(EN_MODEL)
                
                if DROPOUT_SIZE[0] > 0: EN_MODEL = SpatialDropout1D(DROPOUT_SIZE[0])(EN_MODEL)
                for layer in range(GRU_LAYERS-1):
                    EN_MODEL = Bidirectional(GRU(GRU_SIZE,
                                                  return_sequences=True, 
                                                  recurrent_dropout=DROPOUT_SIZE[1]))(EN_MODEL)
                EN_MODEL = Bidirectional(GRU(GRU_SIZE,
                                              return_sequences=True,
                                              recurrent_dropout=DROPOUT_SIZE[1]))(EN_MODEL)
    
                # -------------- ZH MODEL CREATION --------------
                ZH_INPUT = Input(shape=(ZH_SENTENCE_SIZE*2,))
                ZH_MODEL = Reshape((-1,2,ZH_SENTENCE_SIZE))(ZH_INPUT)
                ZH_MODEL = Embedding(zh_vocab_size,
                                EMBED_SIZE,
                                input_shape=(2,ZH_SENTENCE_SIZE),
                                trainable=True)(ZH_MODEL)
                ZH_MODEL = Reshape((2,ZH_SENTENCE_SIZE*EMBED_SIZE,))(ZH_MODEL)
                
                if DROPOUT_SIZE[0] > 0: ZH_MODEL = SpatialDropout1D(DROPOUT_SIZE[0])(ZH_MODEL)
                for layer in range(GRU_LAYERS-1):
                    ZH_MODEL = Bidirectional(GRU(GRU_SIZE,
                                                  return_sequences=True, 
                                                  recurrent_dropout=DROPOUT_SIZE[1]))(ZH_MODEL)
                ZH_MODEL = Bidirectional(GRU(GRU_SIZE,
                                              return_sequences=True,
                                              recurrent_dropout=DROPOUT_SIZE[1]))(ZH_MODEL)
                    
                # -------------- MERGE MODEL --------------
                merged = Concatenate(1)([EN_MODEL,ZH_MODEL])
                merged = GRU(GRU_SIZE,
                              return_sequences=True,
                              recurrent_dropout=DROPOUT_SIZE[1])(merged)
                merged = Flatten()(merged)
                merged = Dense(3, activation='softmax')(merged)

                model = Model(inputs=[EN_INPUT,ZH_INPUT], outputs=merged)
                model.compile(optimizer=optimizer, loss=loss,metrics=metrics)

                # -------------- Training the model --------------
                model.fit([EN_X_train,ZH_X_train], y_train,
                          epochs=epochs,
                          batch_size=batch_size,
                          validation_data=([EN_X_test,ZH_X_test], y_test),
                          callbacks=callbacks,
                          class_weight=class_weights)

Creating BiGRU-50E-1x100G-(0, 0.75)Dropout-1574833242.4031265.hdf5


C:\Users\kevin\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 192223 samples, validate on 64087 samples
Epoch 1/200
  2048/192223 [..............................] - ETA: 8:13 - loss: 1.0551 - accuracy: 0.5420 

C:\Users\kevin\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.224635). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


 55296/192223 [=======>......................] - ETA: 42s - loss: 0.7137 - accuracy: 0.6881